# Spart-part Deduplicate

## Project Overview

Spare-Part Deduplication ด้วย Hybrid RAG และ Graph-based Grouping

โปรเจค Spare-Part Deduplication มีวัตถุประสงค์เพื่อแก้ไขปัญหา material / spare-part ที่ซ้ำกันแต่มีหลายรหัส (PK) ซึ่งเกิดจากการบันทึกข้อมูลจากหลายแหล่ง หลายโรงงาน หรือหลายช่วงเวลา ทำให้ material ที่เป็นชิ้นส่วนเดียวกัน ถูกสร้างเป็นหลาย PK ส่งผลต่อความซับซ้อนของระบบจัดซื้อ คลังสินค้า และการวิเคราะห์ข้อมูลในภาพรวม

แนวคิดและแนวทางแก้ไข (Solution Approach)

โซลูชันของโปรเจคนี้ใช้แนวคิด Retrieval-Augmented Generation (RAG) ในรูปแบบ Hybrid Search เพื่อดึง candidate ที่มีความเป็นไปได้สูงว่าจะเป็น material เดียวกัน จากนั้นใช้ Large Language Model (LLM) เป็นตัวตัดสินเชิงความหมาย (semantic reasoning) ว่า candidate ใดควรถูกจัดเป็น duplicate

ระบบแบ่งการทำงานออกเป็น 3 ขั้นตอนหลัก:

1. Hybrid Search (Candidate Retrieval)

ในขั้นตอนแรก ระบบจะดึง candidate material จากฐานข้อมูล โดยใช้การค้นหาแบบผสม (Hybrid Search) ประกอบด้วย:

Semantic Search (Vector Search)
ใช้ embedding เพื่อจับความหมายของคำอธิบาย material แม้ข้อความจะไม่ตรงกันทุกคำ

Keyword Search (BM25 / Full-text search)
ใช้สำหรับจับ keyword สำคัญ เช่น ประเภทของชิ้นส่วน รหัสรุ่น หรือคำทางเทคนิค

Trigram Fuzzy Matching
ใช้ตรวจจับความคล้ายของข้อความในเชิงตัวอักษร เหมาะกับกรณีสะกดต่างกันเล็กน้อยหรือมีสัญลักษณ์พิเศษ

ผลลัพธ์จากแต่ละวิธีจะถูก normalize และถ่วงน้ำหนัก (weighted scoring) เพื่อคัดเลือก candidate ที่มีแนวโน้มสูงสุดสำหรับขั้นตอนถัดไป

2. LLM-based Grouping (Duplicate Decision)

เมื่อได้ candidate จาก Hybrid Search แล้ว ระบบจะส่งข้อมูลเข้าสู่ LLM เพื่อทำการวิเคราะห์เชิงความหมายและบริบท โดยให้ LLM ตัดสินใจว่า:

material ใดเป็น duplicate ของกันและกัน

material ใดควรถูกแยกเป็นคนละกลุ่ม

LLM จะช่วยจัดกลุ่ม material ที่มีความหมายเหมือนกัน แม้รูปแบบคำอธิบายหรือโครงสร้างข้อมูลจะแตกต่างกัน

3. Graph-based Connectivity Grouping

ผลลัพธ์จาก LLM จะถูกนำไปสร้างเป็น Graph โดย:

material แต่ละ PK คือ node

ความสัมพันธ์ว่า “เป็น duplicate กัน” คือ edge

ระบบจะใช้แนวคิด connected components เพื่อรวมกลุ่ม duplicate ให้ครบถ้วน เช่น:

ถ้า

A = B

B = C

เมื่อนำเข้าสู่ graph จะได้กลุ่มเดียวกันคือ
A = B = C

วิธีนี้ช่วยให้ได้ duplicate group ที่สมบูรณ์ แม้ความสัมพันธ์จะไม่ได้ถูกค้นพบทั้งหมดในครั้งเดียว

diagram flow: (RAG search → LLM → Graph)

ผลลัพธ์ที่คาดหวัง

ลดจำนวน PK ที่ซ้ำซ้อนในระบบ

เพิ่มความถูกต้องของ master data

ลดต้นทุนจากการจัดซื้อซ้ำหรือการจัดเก็บชิ้นส่วนซ้ำ

วางรากฐานสำหรับ data governance และ master data management ในระยะยาว

## DB (RAG)

In [ ]:
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev,public -c client_encoding=UTF8"  # Set default schema and UTF-8 encoding
}

In [ ]:
import psycopg2
import psycopg2.extras
from pgvector.psycopg2 import register_vector
import numpy as np
from typing import List, Dict, Any, Optional, Tuple
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# ============================================================
# Configuration
# ============================================================
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev,public -c client_encoding=UTF8"  # Set default schema and UTF-8 encoding
}

TABLE_NAME = "spare_part_test"
SCHEMA_NAME = "apm_dev"
EMBEDDING_DIM = 1024  # bge-m3 dimension

# ============================================================
# Database Connection
# ============================================================
def get_connection():
    """Get database connection with pgvector support"""
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_client_encoding('UTF8')  # Ensure UTF-8 encoding for Thai text
    register_vector(conn)
    return conn

# ============================================================
# 1. CREATE TABLE
# ============================================================
def create_spare_part_table(drop_if_exists: bool = False):
    """
    Create spare_part_test table with all required fields
    
    Args:
        drop_if_exists: If True, drop existing table before creating
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    try:
        # Drop table if requested
        if drop_if_exists:
            print(f"⚠️  Dropping existing table {SCHEMA_NAME}.{TABLE_NAME}...")
            cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_NAME}.{TABLE_NAME} CASCADE")
        
        # Create table
        create_sql = f"""
        CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
            -- Primary Key
            pk_plant_matnum TEXT PRIMARY KEY,
            
            -- Basic Info
            material_type TEXT,
            plant TEXT,
            material_number TEXT,
            plant_id TEXT,
            plant_description TEXT,
            uom TEXT,
            
            -- Raw Description Fields
            material_description TEXT,
            po_text TEXT,
            
            -- ============================================================
            -- SEMANTIC SEARCH Fields
            -- ============================================================
            raw_combine_for_embedding TEXT,
            clean_combine_for_embedding TEXT,
            embedding vector({EMBEDDING_DIM}),
            
            -- ============================================================
            -- KEYWORD SEARCH Fields (BM25)
            -- ============================================================
            raw_combine_for_keyword TEXT,
            clean_combine_for_keyword TEXT,
            tsv tsvector,
            
            -- Extracted Keywords/Codes
            keyword_codes TEXT[],
            keyword_old_codes TEXT[],
            keyword_list TEXT[],
            
            -- Metadata
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
        print(f"📦 Creating table {SCHEMA_NAME}.{TABLE_NAME}...")
        cur.execute(create_sql)
        
        # Enable required extensions
        print("🔧 Ensuring extensions are enabled...")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
        cur.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm")
        
        # Create indexes
        print("🔍 Creating indexes...")
        
        # A) Vector index for semantic search
        cur.execute(f"""
            DO $$
            BEGIN
                IF NOT EXISTS (
                    SELECT 1 FROM pg_indexes 
                    WHERE schemaname='{SCHEMA_NAME}' 
                    AND tablename='{TABLE_NAME}' 
                    AND indexname='idx_{TABLE_NAME}_embedding'
                ) THEN
                    CREATE INDEX idx_{TABLE_NAME}_embedding 
                    ON {SCHEMA_NAME}.{TABLE_NAME} 
                    USING ivfflat (embedding vector_cosine_ops)
                    WITH (lists = 100);
                END IF;
            END
            $$;
        """)
        
        # B) Full-text search index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_tsv 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin(tsv);
        """)
        
        # C) Trigram index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_pk_trgm 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (pk_plant_matnum gin_trgm_ops);
        """)
        
        # D) Array indexes for keyword matching
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_codes 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_codes);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_list 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_list);
        """)
        
        # E) Basic indexes
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_material_type 
            ON {SCHEMA_NAME}.{TABLE_NAME} (material_type);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_uom 
            ON {SCHEMA_NAME}.{TABLE_NAME} (uom);
        """)
        
        # Create trigger for auto-updating tsv
        print("⚙️  Creating trigger for tsvector auto-update...")
        cur.execute(f"""
            CREATE OR REPLACE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger() 
            RETURNS TRIGGER AS $$
            BEGIN
                NEW.tsv := to_tsvector('simple', COALESCE(NEW.clean_combine_for_keyword, ''));
                NEW.updated_at := CURRENT_TIMESTAMP;
                RETURN NEW;
            END;
            $$ LANGUAGE plpgsql;
            
            DROP TRIGGER IF EXISTS tsv_update_trigger ON {SCHEMA_NAME}.{TABLE_NAME};
            
            CREATE TRIGGER tsv_update_trigger 
            BEFORE INSERT OR UPDATE ON {SCHEMA_NAME}.{TABLE_NAME}
            FOR EACH ROW 
            EXECUTE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger();
        """)
        
        conn.commit()
        print(f"✅ Table {SCHEMA_NAME}.{TABLE_NAME} created successfully!")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error creating table: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Create Table")
print("="*60)
# get_connection()
# create_spare_part_table(drop_if_exists=True)  # Uncomment to run

In [ ]:
# ============================================================
# 2. EMBEDDER CLASS (BGE-M3)
# ============================================================
class BGE_M3_Embedder:
    """
    BGE-M3 Embedding Model Wrapper
    """
    def __init__(self, model_name: str = "BAAI/bge-m3", device: str = None):
        """
        Initialize BGE-M3 model
        
        Args:
            model_name: Model identifier (default: BAAI/bge-m3)
            device: 'cuda', 'cpu', or None (auto-detect)
        """
        print(f"🔄 Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name, device=device)
        self.dim = 1024  # bge-m3 output dimension
        print(f"✅ Model loaded! Dimension: {self.dim}")
    
    def embed_texts(
        self, 
        texts: List[str], 
        batch_size: int = 32,
        show_progress: bool = True
    ) -> List[List[float]]:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            batch_size: Batch size for encoding
            show_progress: Show progress bar
            
        Returns:
            List of embedding vectors (each is list of floats)
        """
        if not texts:
            return []
        
        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            convert_to_numpy=True,
            normalize_embeddings=True  # Recommended for bge-m3
        )
        
        return [emb.tolist() for emb in embeddings]
    
    def embed_single(self, text: str) -> List[float]:
        """Embed a single text (convenience method)"""
        return self.embed_texts([text], show_progress=False)[0]

# Test
print("="*60)
print("TEST: BGE-M3 Embedder")
print("="*60)
# embedder = BGE_M3_Embedder()  # Uncomment to test
# test_emb = embedder.embed_single("BEARING BALL TEST")
# print(f"Embedding dim: {len(test_emb)}")
# print(f"First 5 values: {test_emb[:5]}")

In [ ]:
# ============================================================
# 3. INSERT DATA WITH EMBEDDINGS
# ============================================================
def insert_spare_parts(
    df: pd.DataFrame, 
    embedder: BGE_M3_Embedder, 
    batch_size: int = 128,
    start_from: int = 0
):
    """
    Insert spare part data into database with embeddings
    
    Required DataFrame columns:
        - pk_plant_matnum (PRIMARY KEY)
        - material_type, plant, material_number, plant_id, plant_description, uom
        - material_description, po_text
        - raw_combine_for_embedding, clean_combine_for_embedding
        - raw_combine_for_keyword, clean_combine_for_keyword
        - keyword_codes (list), keyword_old_codes (list), keyword_list (list)
    
    Args:
        df: DataFrame with spare part data
        embedder: BGE_M3_Embedder instance
        batch_size: Number of records to process per batch
        start_from: Skip first N records (for resuming)
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    # Normalize column names to lowercase
    df = df.copy()
    df.columns = df.columns.str.lower()
    
    # Required columns check
    required_cols = [
        'pk_plant_matnum', 'material_type', 'plant', 'material_number', 
        'plantid', 'plant_description', 'uom', 'material_description', 'text',
        'raw_combine_for_embedding', 'clean_combine_for_embedding',
        'raw_combine_for_keyword', 'clean_combine_for_keyword',
        'keyword_codes', 'keyword_old_codes', 'keyword_list'
    ]
    
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    
    # Prepare data
    df_to_insert = df[required_cols].copy()
    total_rows = len(df_to_insert)
    
    if start_from > 0:
        df_to_insert = df_to_insert.iloc[start_from:]
        print(f"⏭️  Skipping first {start_from} records...")
    
    print(f"📊 Total records to insert: {len(df_to_insert)}")
    
    insert_sql = f"""
        INSERT INTO {SCHEMA_NAME}.{TABLE_NAME} (
            pk_plant_matnum, material_type, plant, material_number,
            plant_id, plant_description, uom, material_description, po_text,
            raw_combine_for_embedding, clean_combine_for_embedding, embedding,
            raw_combine_for_keyword, clean_combine_for_keyword,
            keyword_codes, keyword_old_codes, keyword_list
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (pk_plant_matnum) 
        DO UPDATE SET
            material_type = EXCLUDED.material_type,
            material_description = EXCLUDED.material_description,
            clean_combine_for_embedding = EXCLUDED.clean_combine_for_embedding,
            clean_combine_for_keyword = EXCLUDED.clean_combine_for_keyword,
            embedding = EXCLUDED.embedding,
            keyword_codes = EXCLUDED.keyword_codes,
            keyword_list = EXCLUDED.keyword_list,
            updated_at = CURRENT_TIMESTAMP
    """
    
    try:
        inserted = 0
        failed = 0
        
        # Process in batches
        for i in tqdm(range(0, len(df_to_insert), batch_size), desc="Inserting batches"):
            batch_df = df_to_insert.iloc[i:i+batch_size]
            
            # Generate embeddings for this batch
            texts_to_embed = batch_df['clean_combine_for_embedding'].fillna("").tolist()
            embeddings = embedder.embed_texts(texts_to_embed, show_progress=False)
            
            # Prepare batch data
            batch_data = []
            for idx, (_, row) in enumerate(batch_df.iterrows()):
                # Handle column name variations
                plant_id = row.get('plant_id') or row.get('plantid', '')
                po_text = row.get('po_text') or row.get('text', '')
                
                batch_data.append((
                    row['pk_plant_matnum'],
                    row['material_type'],
                    row['plant'],
                    row['material_number'],
                    plant_id,
                    row['plant_description'],
                    row['uom'],
                    row['material_description'],
                    po_text,
                    row['raw_combine_for_embedding'],
                    row['clean_combine_for_embedding'],
                    embeddings[idx],  # embedding vector
                    row['raw_combine_for_keyword'],
                    row['clean_combine_for_keyword'],
                    row['keyword_codes'] if isinstance(row['keyword_codes'], list) else [],
                    row['keyword_old_codes'] if isinstance(row['keyword_old_codes'], list) else [],
                    row['keyword_list'] if isinstance(row['keyword_list'], list) else []
                ))
            
            # Execute batch insert
            try:
                psycopg2.extras.execute_batch(cur, insert_sql, batch_data, page_size=batch_size)
                conn.commit()
                inserted += len(batch_data)
            except Exception as e:
                conn.rollback()
                failed += len(batch_data)
                print(f"\n❌ Batch {i//batch_size + 1} failed: {e}")
                continue
        
        print(f"\n✅ Insertion complete!")
        print(f"   Inserted: {inserted} records")
        if failed > 0:
            print(f"   Failed: {failed} records")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error during insertion: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Create Table & Insert Data")
print("="*60)

import time

# # # Time: Create table
# print("\n🏗️  Creating table...")
# start_time_table = time.time()
# create_spare_part_table(drop_if_exists=True)
# time_table = time.time() - start_time_table
# print(f"✅ Table created in {time_table:.2f} seconds\n")

# # Time: Insert data with embeddings
# print("📥 Inserting data with embeddings...")
# start_time_insert = time.time()
# embedder = BGE_M3_Embedder()
# insert_spare_parts(sample_500, embedder, batch_size=32)
# time_insert = time.time() - start_time_insert
# print(f"✅ Data inserted in {time_insert:.2f} seconds\n")

# print(f"⏱️  Total time: {time_table + time_insert:.2f} seconds")

## Vector Search

In [ ]:
# ============================================================
# 4A. VECTOR SEARCH QUERY (SEMANTIC ONLY)
# ============================================================

def vector_search(
    query_text: str,
    embedder: BGE_M3_Embedder,
    top_k_semantic: int = 100,
    limit: int = 50,
    uom_filter: Optional[str] = None,
    material_type_filter: Optional[str] = None
) -> pd.DataFrame:
    """
    Perform pure vector (semantic) search using embeddings only

    Args:
        query_text: Search query text (for embedding)
        embedder: BGE_M3_Embedder instance
        top_k_semantic: Top K results from vector search
        limit: Final result limit
        uom_filter: Optional UOM filter (e.g., 'PC')
        material_type_filter: Optional material type filter

    Returns:
        DataFrame with search results and semantic scores
    """

    # ------------------------------------------------------------
    # 1) Generate query embedding
    # ------------------------------------------------------------
    query_embedding = embedder.embed_single(query_text)

    conn = get_connection()
    cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

    try:
        # ------------------------------------------------------------
        # 2) Build optional filters
        # ------------------------------------------------------------
        filter_clause = ""
        if uom_filter:
            filter_clause += f" AND uom = '{uom_filter}'"
        if material_type_filter:
            filter_clause += f" AND material_type = '{material_type_filter}'"

        # ------------------------------------------------------------
        # 3) Vector search SQL
        # ------------------------------------------------------------
        search_sql = f"""
        WITH semantic_search AS (
            SELECT
                pk_plant_matnum,
                material_description,
                uom,
                (1 - (embedding <=> %s::vector)) AS semantic_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE embedding IS NOT NULL
                {filter_clause}
            ORDER BY embedding <=> %s::vector
            LIMIT {top_k_semantic}
        ),

        normalized_scores AS (
            SELECT
                pk_plant_matnum,
                material_description,
                uom,
                semantic_score,
                CASE
                    WHEN semantic_score > 0
                        THEN semantic_score / MAX(semantic_score) OVER ()
                    ELSE 0
                END AS semantic_norm
            FROM semantic_search
        )

        SELECT
            pk_plant_matnum,
            material_description,
            uom,
            semantic_norm AS final_score,
            ROUND(semantic_norm::numeric, 3) AS semantic_score
        FROM normalized_scores
        ORDER BY final_score DESC
        LIMIT {limit};
        """

        # ------------------------------------------------------------
        # 4) Execute query
        # ------------------------------------------------------------
        cur.execute(search_sql, (
            query_embedding,
            query_embedding
        ))

        results = cur.fetchall()
        df_results = pd.DataFrame(results)
        return df_results

    except Exception as e:
        print(f"❌ Vector search error: {e}")
        raise

    finally:
        cur.close()
        conn.close()


### Evaluate vector search

In [ ]:
# ============================================================
# EVALUATION: VECTOR SEARCH QUALITY
# ============================================================

def evaluate_vector_search_quality(
    edges_df,
    sample_df,
    embedder,
    top_k=30
):
    """
    ประเมินคุณภาพของ vector (semantic) search
    โดยเช็คว่าเฉลยอยู่ที่ rank เท่าไหร่

    Args:
        edges_df: DataFrame ที่มี Key_mat_1, Key_mat_2 (คู่ที่เป็นเฉลย)
        sample_df: DataFrame ที่มีข้อมูล material ทั้งหมด (sample_500)
        embedder: BGE_M3_Embedder instance
        top_k: จำนวน candidates สูงสุดที่จะค้นหา

    Returns:
        DataFrame สรุปผลการประเมิน
    """

    # ------------------------------------------------------------
    # 1) Build answer map (pk -> list of correct answers)
    # ------------------------------------------------------------
    answer_map = {}
    for _, row in edges_df.iterrows():
        key1 = row['Key_mat_1']
        key2 = row['Key_mat_2']

        if pd.notna(key1) and pd.notna(key2):
            answer_map.setdefault(key1, []).append(key2)
            answer_map.setdefault(key2, []).append(key1)

    print(f"📊 Total PKs with answers: {len(answer_map)}")
    print(f"🔍 Starting VECTOR search evaluation")
    print(f"⏱️  This may take a few minutes...\n")

    results = []

    # ------------------------------------------------------------
    # 2) Helper: case-insensitive column getter
    # ------------------------------------------------------------
    def get_col(rec, col_name):
        col_lower = col_name.lower()
        for col in rec.index:
            if col.lower() == col_lower:
                return rec[col]
        return ''

    # ------------------------------------------------------------
    # 3) Evaluate each PK
    # ------------------------------------------------------------
    for pk, answers in tqdm(answer_map.items(), desc="Evaluating PKs (Vector Search)"):
        record_rows = sample_df[sample_df['pk_plant_matnum'] == pk]

        if len(record_rows) == 0:
            continue

        record = record_rows.iloc[0]

        query_text = get_col(record, 'clean_combine_for_embedding')
        material_desc = get_col(record, 'material_description')

        if not query_text or pd.isna(query_text):
            continue

        try:
            # ----------------------------------------------------
            # Vector search
            # ----------------------------------------------------
            candidates = vector_search(
                query_text=query_text,
                embedder=embedder,
                limit=top_k
            )

            candidate_pks = candidates['pk_plant_matnum'].tolist()

            # ----------------------------------------------------
            # Find ranks of correct answers
            # ----------------------------------------------------
            found_ranks = []
            for answer_pk in answers:
                if answer_pk in candidate_pks:
                    rank = candidate_pks.index(answer_pk) + 1
                    found_ranks.append(rank)

            if found_ranks:
                best_rank = min(found_ranks)
                status = "✅ Found"
            else:
                best_rank = None
                status = "❌ Not Found"

            results.append({
                'pk': pk,
                'material_description': material_desc[:50] if material_desc else '',
                'num_answers': len(answers),
                'answers': ', '.join(answers),
                'found_ranks': ', '.join(map(str, found_ranks)) if found_ranks else 'N/A',
                'best_rank': best_rank,
                'status': status
            })

        except Exception as e:
            print(f"\n⚠️  Error processing {pk}: {e}")
            continue

    # ------------------------------------------------------------
    # 4) Results DataFrame
    # ------------------------------------------------------------
    results_df = pd.DataFrame(results)
    return results_df


In [ ]:
print("="*80)
print("EVALUATION: Vector Search Quality Assessment")
print("="*80)

import time
start_time_eval = time.time()

embedder = BGE_M3_Embedder()

evaluation_results_vec = evaluate_vector_search_quality(
    edges_df=sample100,
    sample_df=sample_500,
    embedder=embedder,
    top_k=50
)

time_eval = time.time() - start_time_eval

print(f"\n✅ Evaluation completed in {time_eval:.2f} seconds")
print(f"⚡ Average time per query: {time_eval/len(evaluation_results_vec):.3f} seconds\n")


In [ ]:
# ============================================================
# SUMMARY STATISTICS: VECTOR SEARCH
# ============================================================

print("\n" + "="*80)
print("📈 SUMMARY STATISTICS (VECTOR SEARCH)")
print("="*80)

total = len(evaluation_results_vec)
found = len(evaluation_results_vec[evaluation_results_vec['status'] == '✅ Found'])
not_found = total - found

print(f"Total PKs evaluated: {total}")
print(f"Found in candidates: {found} ({found/total*100:.1f}%)")
print(f"Not found: {not_found} ({not_found/total*100:.1f}%)")

if found > 0:
    found_df = evaluation_results_vec[
        evaluation_results_vec['status'] == '✅ Found'
    ]

    print(f"\nRank Statistics (for found answers):")
    print(f"  Mean Rank: {found_df['best_rank'].mean():.2f}")
    print(f"  Median Rank: {found_df['best_rank'].median():.1f}")
    print(f"  Best Rank: {found_df['best_rank'].min()}")
    print(f"  Worst Rank: {found_df['best_rank'].max()}")

    total_found = len(found_df)

    top_1 = len(found_df[found_df['best_rank'] == 1])
    top_2_3 = len(found_df[(found_df['best_rank'] >= 2) & (found_df['best_rank'] <= 3)])
    top_4_5 = len(found_df[(found_df['best_rank'] >= 4) & (found_df['best_rank'] <= 5)])
    top_6_10 = len(found_df[(found_df['best_rank'] >= 6) & (found_df['best_rank'] <= 10)])
    top_11_20 = len(found_df[(found_df['best_rank'] >= 11) & (found_df['best_rank'] <= 20)])
    top_21_25 = len(found_df[(found_df['best_rank'] >= 21) & (found_df['best_rank'] <= 25)])
    top_26_30 = len(found_df[(found_df['best_rank'] >= 26) & (found_df['best_rank'] <= 30)])
    top_31_40 = len(found_df[(found_df['best_rank'] >= 31) & (found_df['best_rank'] <= 40)])
    over_40 = len(found_df[found_df['best_rank'] > 40])

    print(f"\n📊 Rank Distribution:")
    print(f"  Top 1: {top_1}")
    print(f"  Rank 2–3: {top_2_3}")
    print(f"  Rank 4–5: {top_4_5}")
    print(f"  Rank 6–10: {top_6_10}")
    print(f"  Rank 11–20: {top_11_20}")
    print(f"  Rank 21–25: {top_21_25}")
    print(f"  Rank 26–30: {top_26_30}")
    print(f"  Rank 31–40: {top_31_40}")
    print(f"  Over 40: {over_40}")

    print(
        f"\n✅ Total counted: "
        f"{top_1 + top_2_3 + top_4_5 + top_6_10 + top_11_20 + top_21_25 + top_26_30 + top_31_40 + over_40}"
    )
    print(f"📦 Total rows: {total_found}")

# ============================================================
# CASES WHERE ANSWER NOT FOUND
# ============================================================

print("\n" + "="*80)
print("❌ CASES WHERE ANSWER NOT FOUND (VECTOR SEARCH)")
print("="*80)

not_found_df = evaluation_results_vec[
    evaluation_results_vec['status'] == '❌ Not Found'
]

if len(not_found_df) > 0:
    print(
        not_found_df[['pk', 'material_description', 'answers']]
        .to_string(index=False)
    )
else:
    print("🎉 All answers were found in candidates!")


## LLM-Grouping

### ✅ LLM Material Grouping - UPDATED VERSION

**สิ่งที่ปรับปรุง:**

1. **Output Schema ที่เรียบง่าย** - เอาเฉพาะข้อมูลที่จำเป็น:
   - `status`: "match" หรือ "unmatch" 
   - `matched_pks`: list ของ PKs ที่เป็น duplicate (ไม่รวม query_pk เอง)
   - `summary`: สรุปสั้นๆ
   
2. **Prompt ที่ชัดเจนขึ้น** - บอก LLM ให้ return เฉพาะ PKs ที่ซ้ำกัน ไม่ต้องส่ง analysis แต่ละตัว

3. **Validation ที่ถูกต้อง** - ใช้ PKs จริงจาก edges.csv:
   - 7560_75060675913234 (ไม่ใช่ 7560_75060670873346)
   - PO11_PO06131335013

In [ ]:
import google.genai as genai

In [ ]:
# ============================================================
# IMPROVED LLM Function with Step-by-Step Reasoning
# ============================================================

from pydantic import BaseModel, Field
from typing import List

class MatchedMaterial(BaseModel):
    """Material ที่ตรงกับ query"""
    pk: str = Field(description="PK ของ candidate ที่เป็น duplicate")
    confidence: float = Field(description="ความมั่นใจ 0.0-1.0", ge=0.0, le=1.0)
    reason: str = Field(description="เหตุผลที่ตรงกัน")

class ImprovedMaterialGroupingResult(BaseModel):
    """ผลลัพธ์การจัดกลุ่ม materials แบบปรับปรุง"""
    query_pk: str
    status: str  # "match" หรือ "unmatch"
    matched_materials: List[MatchedMaterial] = Field(default_factory=list)
    step_by_step_reasoning: str  # Reasoning แบบทีละขั้นตอน
    summary: str

def improved_group_materials_with_llm(
    query_pk: str,
    query_description: str,
    query_full_text: str,
    candidates: pd.DataFrame,
    few_shot_examples: List[dict],
    model: str = "gemini-3-flash-preview",
    min_confidence: float = 0.7
) -> ImprovedMaterialGroupingResult:
    """
    ปรับปรุง LLM grouping ด้วย step-by-step reasoning และ few-shot examples
    """
    
    # เตรียม client
    client = genai.Client(
        vertexai={
            "project": "prj-service-mlops",
            "location": "global"
        }
    )
    
    # เตรียม candidates
    candidate_list = []
    for idx, row in candidates.iterrows():
        candidate_list.append({
            "pk": row['pk_plant_matnum'],
            "description": row['material_description'],
            "full_text": row.get('clean_combine_for_keyword', row['material_description'])[:500]
        })
    
    # สร้าง Few-Shot Examples section
    examples_text = ""
    for i, ex in enumerate(few_shot_examples, 1):
        examples_text += f"""
### Example {i}: Materials that ARE Duplicates

**Query Material:**
- PK: {ex['query_pk']}
- Description: {ex['query_desc']}

**Candidate Material (DUPLICATE):**
- PK: {ex['answer_pk']}
- Description: {ex['answer_desc']}

**Why they are duplicates:** Despite different wording, they refer to the same physical product. Key identifiers and specifications match.

---
"""
    
    # สร้าง prompt ใหม่
    prompt = f"""# Advanced Material Deduplication with Step-by-Step Analysis

You are an expert in industrial spare parts identification. Your task is to identify which candidate materials are EXACT DUPLICATES of the query material using systematic analysis.

## 📋 Few-Shot Examples (Learn from these):
{examples_text}

## 🎯 Current Task:

### Query Material (Find duplicates of this):
- **PK**: {query_pk}
- **Description**: {query_description}
- **Full Text**: {query_full_text[:800]}

### Candidate Materials (Analyze each one):
"""
    
    for i, cand in enumerate(candidate_list, 1):
        prompt += f"""
**Candidate #{i}:**
- PK: {cand['pk']}
- Description: {cand['description']}
- Full Text: {cand['full_text']}
"""
    
    prompt += """

## 🔍 Analysis Framework (Follow these steps):

**Step 1: Extract Key Identifiers**
- Part numbers, model codes, manufacturer codes
- Size/dimensions (numbers with units like mm, cm, inch)
- Material type (steel, plastic, rubber, etc.)
- Capacity/rating (voltage, pressure, flow rate)

**Step 2: Compare Technical Specifications**
- Are dimensions identical or equivalent?
- Same material composition?
- Same technical ratings?
- Same functionality?

**Step 3: Evaluate Semantic Similarity**
- Do descriptions refer to the same product?
- Are they used for the same purpose?
- Same application area?

**Step 4: Determine Confidence Level**
- **High (0.9-1.0)**: Identical part numbers AND specs
- **Medium-High (0.8-0.89)**: Same specs, similar codes
- **Medium (0.7-0.79)**: Same specs, different codes
- **Below 0.7**: Uncertain or different materials

**Step 5: Make Final Decision**
- Only include matches with confidence ≥ 0.7
- When uncertain, be conservative (don't match)

## ✅ Matching Rules:

**MATCH if:**
- Same part number/model code (even with plant prefix)
- Same specifications (size, material, capacity)
- Different only in: plant code, vendor ID, packaging info

**DO NOT MATCH if:**
- Different part numbers AND different specs
- Different sizes or capacities
- Different materials or types
- One is accessory, another is main product
- Similar but not identical

## 📊 Output Format:

For each duplicate found, provide:
1. **pk**: The candidate PK
2. **confidence**: 0.7-1.0 (≥0.7 to include)
3. **reason**: Brief explanation (30-50 words)

Include **step_by_step_reasoning** showing your analysis process.

## ⚠️ Important Notes:

- Be systematic - analyze ALL candidates
- Be conservative - only high confidence matches
- Ignore the first candidate if it's the query itself (same PK)
- Focus on TECHNICAL match, not just keyword similarity
- Different plants can have same material

Analyze carefully and provide your results."""

    # เรียก LLM
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=ImprovedMaterialGroupingResult,
            temperature=0.1,  # เพิ่มเล็กน้อยเพื่อความยืดหยุ่น
            top_p=0.95
        )
    )
    
    # Parse response
    result = ImprovedMaterialGroupingResult.model_validate_json(response.text)
    
    # Filter by confidence threshold
    filtered_matches = [
        m for m in result.matched_materials 
        if m.confidence >= min_confidence
    ]
    
    result.matched_materials = filtered_matches
    if not filtered_matches:
        result.status = "unmatch"
    
    return result

print("✅ Improved LLM function created")
print(f"   Model: gemini-3-flash-preview")
print(f"   Min confidence: 0.7")
print(f"   Features: Step-by-step reasoning + Few-shot examples")

In [ ]:
# ============================================================
# ทดสอบ Improved LLM กับ Test Cases
# ============================================================

print("\n" + "="*80)
print("🧪 TESTING IMPROVED LLM (gemini-3-flash-preview)")
print("="*80)

improved_llm_results = []

for i, test_result in enumerate(test_results, 1):
    print(f"\n[{i}/{len(test_results)}] Case {test_result['case_number']}: {test_result['query_pk'][:40]}...")
    
    # ดึง query info
    query_pk = test_result['query_pk']
    query_description = test_result['query_description']
    
    query_record = sample_500[sample_500['pk_plant_matnum'] == query_pk].iloc[0]
    query_full_text = query_record.get('clean_combine_for_keyword', query_description)
    
    # แปลง candidates เป็น DataFrame
    candidates_df = pd.DataFrame(test_result['candidates'])
    
    try:
        # ส่งให้ Improved LLM
        print(f"   📤 Sending to improved LLM...")
        
        improved_result = improved_group_materials_with_llm(
            query_pk=query_pk,
            query_description=query_description,
            query_full_text=query_full_text,
            candidates=candidates_df,
            few_shot_examples=selected_examples,
            model="gemini-3-flash-preview",
            min_confidence=0.7
        )
        
        # เก็บผลลัพธ์
        result_dict = {
            "case_number": test_result['case_number'],
            "query_pk": test_result['query_pk'],
            "has_answer": test_result['has_answer'],
            "answer_pk": test_result['answer_pk'],
            "answer_in_candidates": test_result.get('answer_in_candidates'),
            "answer_rank": test_result.get('answer_rank'),
            "llm_response": improved_result,
            "matched_pks": [m.pk for m in improved_result.matched_materials]
        }
        
        # แสดงผลลัพธ์
        if improved_result.status == "match" and improved_result.matched_materials:
            print(f"   ✅ Status: {improved_result.status}")
            print(f"   📋 Found {len(improved_result.matched_materials)} matches:")
            for m in improved_result.matched_materials[:3]:
                print(f"      - {m.pk[:35]}... (conf: {m.confidence:.2f})")
        else:
            print(f"   ❌ Status: {improved_result.status} (no matches)")
        
        # เช็คความถูกต้อง
        if test_result['has_answer']:
            llm_found_answer = test_result['answer_pk'] in result_dict['matched_pks']
            result_dict['llm_correct'] = llm_found_answer
            
            if llm_found_answer:
                # หา confidence ของ answer
                answer_match = next((m for m in improved_result.matched_materials 
                                    if m.pk == test_result['answer_pk']), None)
                if answer_match:
                    print(f"   🎯 CORRECT! Found answer (confidence: {answer_match.confidence:.2f})")
            else:
                print(f"   ⚠️ MISSED answer: {test_result['answer_pk'][:30]}...")
        else:
            result_dict['llm_correct'] = None
        
        improved_llm_results.append(result_dict)
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        result_dict = {
            "case_number": test_result['case_number'],
            "query_pk": test_result['query_pk'],
            "has_answer": test_result['has_answer'],
            "answer_pk": test_result['answer_pk'],
            "error": str(e)
        }
        improved_llm_results.append(result_dict)
        continue

print(f"\n{'='*80}")
print(f"✅ Improved LLM Complete: {len(improved_llm_results)}/{len(test_results)} cases")
print("="*80)